In [6]:
#import all dependencies

import pandas as pd
import time
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials


In [3]:
#create variable browser, and set its path to where you have the chrome webdriver .exe file

browser = webdriver.Chrome(executable_path="/Users/analo/Downloads/chromedriver_win32/chromedriver")

C:\Users\analo\AppData\Local\Temp\ipykernel_25652\441148923.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path="/Users/analo/Downloads/chromedriver_win32/chromedriver")


In [4]:
#creating the variable 'url' and set its value to the link containing the financial times article

url = 'https://www.ft.com/content/2e45727c-0c2a-4636-95d0-4889969cfca8'

In [7]:
#see if the browser works. if it works, a new chrome (or whatever browser you use) opens up the ft article, and scrolls to the bottom of the page

browser.get(url)
browser.execute_script("window.scrollTo(1,20000)")
time.sleep(2)

In [8]:
#store the content of the page in the variable 'html' using the page source attribute of the chrome web driver
html = browser.page_source

In [9]:
#store the content in an html file called 'ft_report'

with open('ft_report.html', 'w+', encoding='utf-8', errors='backslashreplace') as f:
    f.write(html)

In [10]:
#next, read the value of the ft_report file and store in the variable 'ft_data'

with open('ft_report.html', 'r', encoding='utf-8', errors='backslashreplace') as f:
    ft_data = f.read()

In [11]:
#now, we extract the container 'tbody' that contains all the rows in the ft article table containing the data we need
#we utilize the 'find_element' method of the chrome web driver (called browser in this case), and we use it to find the XPATH of the table rows elements 
#we also create an empty list 'data' in this list we will write a for loop and append all the row elements

tbody = browser.find_element(By.XPATH, '//*[@id="site-content"]/div[3]/div[3]/div/div[1]/div/div[1]/table/tbody')

data = []

In [12]:
#now, we write the for loop to iterate through tbody, and store each row element in the list 'data'

for tr in tbody.find_elements(By.XPATH, '//tr'):
    row = [item.text for item in tr.find_elements(By.XPATH, './/td')]
    data.append(row)

In [13]:
#next, we do the same thing as above, but this time we're interested in the column headers contained in the 'thead' container
#we also create an empty list 'data_col' to store all the column header names using another forloop

thead = browser.find_element(By.XPATH, '//*[@id="site-content"]/div[3]/div[3]/div/div[1]/div/div[1]/table/thead')

data_col = []

In [14]:
#this for loop iterates through the 'thead' variable and stores all the header names

for tr in thead.find_elements(By.XPATH, '//tr'):
    col = [item.text for item in tr.find_elements(By.XPATH, './/th')]
    data_col.append(col)

In [15]:
#we need only the first element in data_col

data_col = data_col[0]

In [16]:
#for 'data' we need the first 50 columns starting from index 1

data = data[1:51]

In [17]:
#now, we create the dataframe using 'data' as the row values, and 'data_col' as the column headers
#we store this dataframe in a new variable called 'ft_data'

ft_data = pd.DataFrame(data, columns = data_col)

In [18]:
#next, we check to make sure our dataframe is as expected by returning only the 'head'

ft_data.head()

,Rank,Company name,Brand,Country,Sector,Absolute growth rate (%),"Compound annual growth rate (CAGR, %)",Revenues 2021 ($mn),Revenues 2018 ($mn),Number of employees 2021,Number of employees 2018,Founding year
0,1,AFEX Commodities Exchange Ltd.,AFEX,Nigeria,"Agriculture, Forestry & Fishing","22,076.31",505.289,102,0.5,278,64,2014
1,2,"Moniepoint, Inc. (formerly TeamApt, Inc.)",Moniepoint,Nigeria,"Fintech, Financial Services & Insurance","7,372.70",321.204,62.6,0.8,538,64,2015
2,3,Sokowatch Ltd.,Wasoko,Kenya,E-commerce,"3,901.40",242.035,136.9,3.4,"1,490",104,2016
3,4,Altech,Altech,Democratic Republic of Congo,Energy & Utilities,"3,017.43",214.726,17,0.5,"4,000",800,2013
4,5,The Africaworks Group of Companies,Africaworks,Mauritius,Real estate,"2,314.43",189.027,6.8,0.3,105,12,2017


###To Learn how to push a pandas dataframe into google sheet, please read this brilliant article by *Melvynn Fernandez* https://towardsdatascience.com/using-python-to-push-your-pandas-dataframe-to-google-sheets-de69422508f

###To learn how to authenticate your account to access Google Sheet API's please read this https://docs.gspread.org/en/latest/oauth2.html

In [19]:
#next we create try to upload our dataframe into a googlesheet file to serve as the database sort of
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'maps-project-268611-f4cdd4198cf5.json', scope)

In [ ]:
#finally, we push this dataframe to googlesheet, and we name the sheet that contains our full data 'List'

spreadsheet_key = '12eGR4auYOJlqZE5xjPc6JY6qJdhfr3lvHU4ZhOWv1fU'
wks_name = 'List'
d2g.upload(ft_data, spreadsheet_key, wks_name, credentials=credentials, row_names=True)